<a href="https://colab.research.google.com/github/tranvangaohd1994/STP_2021_2/blob/KhanhNQ/Fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras_applications
!pip install keras_preprocessing

     |████████████████████████████████| 50 kB 2.7 MB/s 


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import vgg16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os

In [1]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
   creating: dataset/Bluebell/
  inflating: dataset/Bluebell/image_0241.jpg  
  inflating: dataset/Bluebell/image_0242.jpg  
  inflating: dataset/Bluebell/image_0243.jpg  
  inflating: dataset/Bluebell/image_0244.jpg  
  inflating: dataset/Bluebell/image_0245.jpg  
  inflating: dataset/Bluebell/image_0246.jpg  
  inflating: dataset/Bluebell/image_0247.jpg  
  inflating: dataset/Bluebell/image_0248.jpg  
  inflating: dataset/Bluebell/image_0249.jpg  
  inflating: dataset/Bluebell/image_0250.jpg  
  inflating: dataset/Bluebell/image_0251.jpg  
  inflating: dataset/Bluebell/image_0252.jpg  
  inflating: dataset/Bluebell/image_0253.jpg  
  inflating: dataset/Bluebell/image_0254.jpg  
  inflating: dataset/Bluebell/image_0255.jpg  
  inflating: dataset/Bluebell/image_0256.jpg  
  inflating: dataset/Bluebell/image_0257.jpg  
  inflating: dataset/Bluebell/image_0258.jpg  
  inflating: dataset/Bluebell/image_0259.jpg  
  inflating: dataset/Bluebell/im

In [4]:
image_path = list(paths.list_images('dataset/'))

random.shuffle(image_path)

In [5]:
labels = [p.split(os.path.sep)[-2] for p in image_path]

le = LabelEncoder()
labels = le.fit_transform(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [6]:
list_image = []
for (j, imagePath) in enumerate(image_path):
  image = load_img(imagePath, target_size=(224,224))
  image = img_to_array(image)
  image = np.expand_dims(image, 0)
  image = imagenet_utils.preprocess_input(image)
  
  list_image.append(image)

list_image = np.vstack(list_image)

In [8]:
import tensorflow as tf
baseModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

fcHead = baseModel.output

fcHead = Flatten(name='flatten')(fcHead)

fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)

fcHead = Dense(17, activation='softmax')(fcHead)

model = model = Model(inputs=baseModel.input, outputs=fcHead)

58900480/58889256 [==============================] - 1s 0us/step


In [9]:
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [10]:
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

aug_test = ImageDataGenerator(rescale=1./255)

In [11]:
for layer in baseModel.layers:
  layer.trainable = False

opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
numOfEpochs = 25
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpochs)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
34/34 [==============================] - 58s 496ms/step - loss: 4.3644 - accuracy: 0.1664 - val_loss: 2.1818 - val_accuracy: 0.3984
Epoch 2/25
34/34 [==============================] - 17s 488ms/step - loss: 2.2810 - accuracy: 0.2886 - val_loss: 1.6582 - val_accuracy: 0.5234
Epoch 3/25
34/34 [==============================] - 17s 488ms/step - loss: 2.0692 - accuracy: 0.3750 - val_loss: 1.1775 - val_accuracy: 0.6758
Epoch 4/25
34/34 [==============================] - 17s 491ms/step - loss: 1.6516 - accuracy: 0.4908 - val_loss: 1.1499 - val_accuracy: 0.6836
Epoch 5/25
34/34 [==============================] - 17s 489ms/step - loss: 1.5368 - accuracy: 0.4972 - val_loss: 0.9278 - val_accuracy: 0.7305
Epoch 6/25
34/34 [==============================] - 17s 488ms/step - loss: 1.4197 - accuracy: 0.5496 - val_loss: 0.8498 - val_accuracy: 0.7266
Epoch 7/25
34/34 [==============================] - 17s 490ms/step - loss: 1.2452 - accuracy: 0.6048 - val_loss: 0.7060 - val_accuracy: 0.7773

In [13]:
for layer in baseModel.layers[15:]:
  layer.trainalbe=True

numOfEpochs=35
opt=SGD(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpochs)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/35
34/34 [==============================] - 18s 502ms/step - loss: 0.5699 - accuracy: 0.8088 - val_loss: 0.2866 - val_accuracy: 0.9062
Epoch 2/35
34/34 [==============================] - 17s 492ms/step - loss: 0.4995 - accuracy: 0.8318 - val_loss: 0.2504 - val_accuracy: 0.9102
Epoch 3/35
34/34 [==============================] - 17s 488ms/step - loss: 0.4616 - accuracy: 0.8401 - val_loss: 0.2789 - val_accuracy: 0.8984
Epoch 4/35
34/34 [==============================] - 17s 488ms/step - loss: 0.4348 - accuracy: 0.8566 - val_loss: 0.2806 - val_accuracy: 0.8984
Epoch 5/35
34/34 [==============================] - 17s 491ms/step - loss: 0.4305 - accuracy: 0.8557 - val_loss: 0.2722 - val_accuracy: 0.8945
Epoch 6/35
34/34 [==============================] - 17s 491ms/step - loss: 0.4756 - accuracy: 0.8373 - val_loss: 0.2653 - val_accuracy: 0.8984
Epoch 7/35
34/34 [==============================] - 17s 492ms/step - loss: 0.4328 - accuracy: 0.8557 - val_loss: 0.2492 - val_accuracy: 0.9023